# Batch pre-computed recommendations

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import os
import sys

import redis
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel
from tqdm.auto import tqdm

load_dotenv()

sys.path.insert(0, "..")

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    run_name: str = "000-first-attempt"
    notebook_persist_dp: str = None
    random_seed: int = 41

    redis_host: str = "localhost"
    redis_port: int = 6379
    redis_key_prefix: str = "output:i2i:"

    batch_recs_fp: str = "data/000-first-attempt/batch_recs.jsonl"

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")
        os.makedirs(self.notebook_persist_dp, exist_ok=True)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

{
  "testing": false,
  "run_name": "000-first-attempt",
  "notebook_persist_dp": "/Users/quy.dinh/frostmourne/recsys-mvp/notebooks/data/000-first-attempt",
  "random_seed": 41,
  "redis_host": "localhost",
  "redis_port": 6379,
  "redis_key_prefix": "output:i2i:",
  "batch_recs_fp": "data/000-first-attempt/batch_recs.jsonl"
}


# Load batch recs into Redis

In [4]:
r = redis.Redis(host=args.redis_host, port=args.redis_port, db=0, decode_responses=True)
assert (
    r.ping()
), f"Redis at {args.redis_host}:{args.port} is not running, please make sure you have started the Redis docker service"

In [5]:
def store_recommendations(file_path: str):
    with open(file_path, "r") as file:
        for line in tqdm(file):
            rec_data = json.loads(line)
            target_item = rec_data["target_item"]
            key = args.redis_key_prefix + target_item
            r.set(
                key,
                json.dumps(
                    {
                        "rec_item_ids": rec_data["rec_item_ids"],
                        "rec_scores": rec_data["rec_scores"],
                    }
                ),
            )


def get_recommendations(target_item):
    key = args.redis_key_prefix + target_item
    rec_data = r.get(target_item)
    if rec_data:
        return json.loads(rec_data)
    return None


def get_example_keys(count=5):
    keys = r.scan_iter(match=args.redis_key_prefix + "*", count=count)
    output = []
    for i, key in enumerate(keys, 1):
        output.append(key)
        if i >= count:
            return output

In [6]:
logger.info(f"Loading batch recs output from {args.batch_recs_fp}...")
store_recommendations(args.batch_recs_fp)

2024-10-10 11:50:39.456 | INFO     | __main__:<module>:1 - Loading batch recs output from data/000-first-attempt/batch_recs.jsonl...


0it [00:00, ?it/s]

In [7]:
get_recommendations(get_example_keys()[0])

{'rec_item_ids': ['B071J4RGZB',
  'B07177727N',
  'B06WGSKRKL',
  'B07ZGJB3WG',
  'B074TRZVYZ',
  'B07KS74Q2X',
  'B07F8K7N51',
  'B07M8ZXQR6',
  'B01A0PVN5A',
  'B08S5YCZ6Q',
  'B00OLOQGAY',
  'B00BQMGW4Y',
  'B084QDTVC6',
  'B01KUP6K3U',
  'B0017QFMJU',
  'B07Q2C9N8W',
  'B01D159YQQ',
  'B0BFFJTZNN',
  'B071GPJS1Q',
  'B01LDWMGNC',
  'B06XWD8QQJ',
  'B002FQJUQ2',
  'B01H2YOA3O',
  'B07MBKDSQ6',
  'B01B1KEERW',
  'B00T57U7V6',
  'B000NJBD90',
  'B07D19QQHW',
  'B07HVHT7XF',
  'B0050SY8UK',
  'B01KUAMCWI',
  'B0095C0I4W',
  'B01EW9XAKQ',
  'B0C2YCG5Y2',
  'B0083CJ2X8',
  'B09918MSTF',
  'B077TQWCFB',
  'B00EZIXEIS',
  'B08JJ37XVW',
  'B072FRJWQH',
  'B07T5QKKVP',
  'B0C2HFCDR3',
  'B0056C2LIG',
  'B0012GUAII',
  'B07JJLLFXJ',
  'B07BF2422D',
  'B07HJSNBBH',
  'B004W4TG5K',
  'B004WLRR4K',
  'B013OW1FJ0',
  'B00002STFD',
  'B07KXPJS16',
  'B000KQQUMQ',
  'B0B1PB5L93',
  'B07H9NSBH3',
  'B0728L4V8P',
  'B0094H8H7I',
  'B001EYUWDG',
  'B01GW3GSVS',
  'B01GHSPTKY',
  'B01J3MKLHC',
  'B0BWN